In [1]:
import numpy as np
from pandas import read_csv

In [2]:
#lstm_data = np.genfromtxt('./sample_data/lstm.csv', delimiter=',', skip_header=True)
lstm_data = read_csv('lstmshib.csv')
lstm_data = lstm_data.drop(['Date'], axis=1)
print(lstm_data)

        High       Low      Open     Close      Volume  Adj Close  \
0   0.000021  0.000020  0.000021  0.000021   421850042   0.000021   
1   0.000022  0.000021  0.000021  0.000022   609855710   0.000022   
2   0.000032  0.000029  0.000029  0.000030  2065365631   0.000030   
3   0.000031  0.000029  0.000030  0.000030  1558119657   0.000030   
4   0.000032  0.000030  0.000030  0.000031  1602354887   0.000031   
5   0.000031  0.000030  0.000031  0.000031  1350071541   0.000031   
6   0.000032  0.000028  0.000031  0.000028  2067544526   0.000028   
7   0.000030  0.000028  0.000028  0.000028  1458694142   0.000028   
8   0.000029  0.000027  0.000028  0.000028   929189862   0.000028   
9   0.000028  0.000025  0.000028  0.000026  1235841144   0.000026   
10  0.000027  0.000024  0.000026  0.000024  2001180593   0.000024   
11  0.000025  0.000023  0.000024  0.000025  1524043647   0.000025   
12  0.000027  0.000025  0.000025  0.000025  1728129985   0.000025   
13  0.000025  0.000021  0.000025  

In [3]:
lstm_data_X = lstm_data.drop(['Close'], axis=1)
lstm_data_y = lstm_data['Close']

In [4]:
look_back = 3 # how many days to look back
batch_size = 5 # size of batches used when training
n_feat = 6 # number of features 
n_target = 2
n_train = 30
n_validation = 5
n_test = 6

In [5]:
lstm_data_X_train = lstm_data_X.iloc[:30,:]
lstm_data_X_val = lstm_data_X.iloc[30:35,:]
lstm_data_X_test = lstm_data_X.iloc[35:50,:]

lstm_data_y_train = lstm_data_y.iloc[:30]
lstm_data_y_val = lstm_data_y.iloc[30:35]
lstm_data_y_test = lstm_data_y.iloc[35:50]
# Convert to numpy arrays
X_train = lstm_data_X_train.to_numpy()
X_val = lstm_data_X_val.to_numpy()
X_test = lstm_data_X_test.to_numpy()
y_train = lstm_data_y_train.to_numpy()
y_val = lstm_data_y_val.to_numpy()
y_test = lstm_data_y_test.to_numpy()

In [6]:
X_test

array([[2.90000000e-05, 2.50000000e-05, 2.60000000e-05, 3.56160726e+09,
        2.60000000e-05, 1.53586280e-01],
       [2.80000000e-05, 2.60000000e-05, 2.60000000e-05, 2.06383842e+09,
        2.70000000e-05, 1.14973180e-01],
       [2.80000000e-05, 2.60000000e-05, 2.70000000e-05, 1.62991388e+09,
        2.70000000e-05, 1.07060620e-01],
       [2.80000000e-05, 2.60000000e-05, 2.70000000e-05, 1.41851644e+09,
        2.60000000e-05, 8.39072810e-02],
       [2.70000000e-05, 2.60000000e-05, 2.60000000e-05, 9.18512393e+08,
        2.70000000e-05, 1.46417220e-01],
       [2.70000000e-05, 2.60000000e-05, 2.70000000e-05, 1.10135640e+09,
        2.60000000e-05, 9.08316200e-02],
       [2.80000000e-05, 2.60000000e-05, 2.70000000e-05, 2.03268282e+09,
        2.70000000e-05, 1.19920160e-01],
       [2.70000000e-05, 2.50000000e-05, 2.70000000e-05, 1.15216252e+09,
        2.60000000e-05, 1.67889080e-01],
       [2.60000000e-05, 2.60000000e-05, 2.60000000e-05, 4.64520286e+08,
        2.60000000e-05, 

In [7]:
y_test

array([2.60e-05, 2.70e-05, 2.70e-05, 2.60e-05, 2.70e-05, 2.60e-05,
       2.70e-05, 2.60e-05, 2.60e-05, 2.60e-05, 2.50e-05, 2.50e-05,
       2.50e-05, 2.50e-05, 2.43e-05])

In [8]:
import keras
from keras.preprocessing.sequence import TimeseriesGenerator
from keras.models import Sequential
from keras.layers import LSTM, Dense, Activation, ThresholdedReLU, MaxPooling2D, Embedding, Dropout
from tensorflow.keras.optimizers import Adam, SGD, RMSprop

In [9]:
train_data_gen = TimeseriesGenerator(X_train, y_train, length=look_back, batch_size=batch_size)
val_data_gen = TimeseriesGenerator(X_val, y_val, length=look_back, batch_size=batch_size)
test_data_gen = TimeseriesGenerator(X_test, y_test, length=look_back, batch_size=batch_size)

In [10]:
# check generator dimensions
for i in range(len(train_data_gen)):
    x, y = train_data_gen[i]
    print(x.shape, y.shape)

(5, 3, 6) (5,)
(5, 3, 6) (5,)
(5, 3, 6) (5,)
(5, 3, 6) (5,)
(5, 3, 6) (5,)
(2, 3, 6) (2,)


In [11]:
model_lstm = Sequential()
model_lstm.add(LSTM(32, input_shape=(look_back, n_feat), return_sequences=True))
model_lstm.add(Dropout(0.1))
model_lstm.add(LSTM(32))
model_lstm.add(Dropout(0.1))
model_lstm.add(Dense(1))
model_lstm.compile(optimizer="adam", loss='mse', metrics=["mse"])
print(model_lstm.summary())

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm (LSTM)                 (None, 3, 32)             4992      
                                                                 
 dropout (Dropout)           (None, 3, 32)             0         
                                                                 
 lstm_1 (LSTM)               (None, 32)                8320      
                                                                 
 dropout_1 (Dropout)         (None, 32)                0         
                                                                 
 dense (Dense)               (None, 1)                 33        
                                                                 
Total params: 13,345
Trainable params: 13,345
Non-trainable params: 0
_________________________________________________________________
None


In [12]:
hist = model_lstm.fit_generator(train_data_gen,
                                        steps_per_epoch=10,
                                        epochs=20,
                                        verbose=1,
                                        validation_data=val_data_gen)

C:\Users\moham\AppData\Local\Temp/ipykernel_11696/192094070.py:1: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  hist = model_lstm.fit_generator(train_data_gen,


Epoch 1/20
10/10 [==============================] - 8s 311ms/step - loss: 0.0027 - mse: 0.0027 - val_loss: 8.1934e-04 - val_mse: 8.1934e-04


In [13]:
pred = model_lstm.predict(test_data_gen)

In [14]:
test_data_gen[0]

(array([[[2.90000000e-05, 2.50000000e-05, 2.60000000e-05, 3.56160726e+09,
          2.60000000e-05, 1.53586280e-01],
         [2.80000000e-05, 2.60000000e-05, 2.60000000e-05, 2.06383842e+09,
          2.70000000e-05, 1.14973180e-01],
         [2.80000000e-05, 2.60000000e-05, 2.70000000e-05, 1.62991388e+09,
          2.70000000e-05, 1.07060620e-01]],
 
        [[2.80000000e-05, 2.60000000e-05, 2.60000000e-05, 2.06383842e+09,
          2.70000000e-05, 1.14973180e-01],
         [2.80000000e-05, 2.60000000e-05, 2.70000000e-05, 1.62991388e+09,
          2.70000000e-05, 1.07060620e-01],
         [2.80000000e-05, 2.60000000e-05, 2.70000000e-05, 1.41851644e+09,
          2.60000000e-05, 8.39072810e-02]],
 
        [[2.80000000e-05, 2.60000000e-05, 2.70000000e-05, 1.62991388e+09,
          2.70000000e-05, 1.07060620e-01],
         [2.80000000e-05, 2.60000000e-05, 2.70000000e-05, 1.41851644e+09,
          2.60000000e-05, 8.39072810e-02],
         [2.70000000e-05, 2.60000000e-05, 2.60000000e-05, 

In [15]:
pred

array([[0.02864908],
       [0.02864908],
       [0.02864908],
       [0.02864908],
       [0.02864908],
       [0.02864908],
       [0.02864908],
       [0.02864908],
       [0.02864908],
       [0.02864908],
       [0.02864908],
       [0.02864908]], dtype=float32)